In [31]:
# Import Statements

from keras.models import *
import keras.backend as K
import cv2
import numpy as np
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Dense, Lambda
import os



def target_category_loss(x, category_index, nb_classes):
    return tf.multiply(x, K.one_hot([category_index], nb_classes))

def normalize(x):
    # utility function to normalize a tensor by its L2 norm
    return x / (K.sqrt(K.mean(K.square(x))) + 1e-5)

def target_category_loss_output_shape(input_shape):
    return input_shape

def grad_cam(input_model, test_img_path, category_index, output_path):
    #Read original image
    original_img = cv2.imread(test_img_path, 1)

    #Resize image to pass through the model (not necessary)
    img = cv2.resize(original_img,(150,150))
    width, height, _ = img.shape

    #Expand dims to create a batch of size 1
    batch_img = np.expand_dims(img,axis=0)

    nb_classes = 2
    target_layer = lambda x: target_category_loss(x, category_index, nb_classes)
    model.add(Lambda(target_layer,
                     output_shape = target_category_loss_output_shape))

    loss = K.sum(model.layers[-1].output)
    conv_output =  model.layers[-4].output
    print(conv_output.shape)
    grads = normalize(K.gradients(loss, conv_output)[0])
    gradient_function = K.function([model.layers[0].input], [conv_output, grads])

    output, grads_val = gradient_function([batch_img])
    print(output.shape,grads_val.shape)
    output, grads_val = output[0, :], grads_val[0, :, :, :]

    weights = np.mean(grads_val, axis = (0, 1))
    cam = np.ones(output.shape[0 : 2], dtype = np.float32)

    for i, w in enumerate(weights):
        cam += w * output[:, :, i]

    cam = cv2.resize(cam, (150, 150))
    cam = np.maximum(cam, 0)
    heatmap = cam / np.max(cam)

    img -= np.min(img)
    img = np.minimum(img, 255)

    cam = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
    cam = np.float32(cam) + np.float32(img)
    cam = 255 * cam / np.max(cam)
    cam = np.uint8(cam)
    cv2.imwrite(output_path,cam)
    return cam, heatmap

In [30]:
# dimensions of our images.
img_width, img_height = 150, 150

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)
    
#Helper functions to define model
def global_average_pooling(x):
    return K.mean(x, axis = (1, 2))

def global_average_pooling_shape(input_shape):
    return input_shape[0:2]

import tensorflow as tf
tf.compat.v1.disable_eager_execution()

#Defining Model
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape,activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Lambda(global_average_pooling, 
              output_shape=global_average_pooling_shape))
model.add(Dense(2, activation = 'softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


#Since we have a trained model, load the model weights
model_path = 'weights/weights.50-0.43.h5' 
model.load_weights(model_path)

#Dog CAM
path = 'data/train/dogs'
images = os.listdir(path)
index = np.random.randint(0,len(images))
img_path = os.path.join(path,images[index])

cam,heatmap = grad_cam(model,img_path,1,'GRADCAM/dog.png')


(None, 7, 7, 64)
(1, 7, 7, 64) (1, 7, 7, 64)
